<a href="https://colab.research.google.com/github/pranao0609/DL_Projects/blob/main/25_Intent_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
data = [
    ("hello", "greeting"),
    ("hi", "greeting"),
    ("hey", "greeting"),
    ("bye", "goodbye"),
    ("see you", "goodbye"),
    ("thanks", "thanks"),
    ("thank you", "thanks"),
    ("what can you do", "query"),
    ("how can you help", "query")
]

texts, labels = zip(*data)
label2idx = {label: idx for idx, label in enumerate(set(labels))}
idx2label = {idx: label for label, idx in label2idx.items()}
y = [label2idx[label] for label in labels]

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts).toarray()

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)


In [29]:
class IntentModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

    def forward(self, x):
        return self.fc(x)

model = IntentModel(X.shape[1], len(label2idx))
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [30]:
for epoch in range(100):
    outputs = model(X_tensor)
    loss = loss_fn(outputs, y_tensor)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 20, Loss: 0.4919
Epoch 40, Loss: 0.0308
Epoch 60, Loss: 0.0050
Epoch 80, Loss: 0.0025
Epoch 100, Loss: 0.0018


In [31]:
def predict_intent(query):
    vec = torch.tensor(vectorizer.transform([query]).toarray(), dtype=torch.float32)
    pred = torch.argmax(model(vec))
    return idx2label[pred.item()]

print("Predicted Intent:", predict_intent("hi there"))
print("Predicted Intent:", predict_intent("what can you do for me"))

Predicted Intent: greeting
Predicted Intent: query
